<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Aggregate Operators

### `aggregate_temporal_period`: temporal aggregation with predefined intervals

Start importing the necessary libraries and initialize a local connection for Client-Side Processing.

In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`


Create the starting Sentinel-2 datacube:

In [2]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2022-01-01", "2022-12-31"]
bands = ["red","green","blue"]

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands
)
s2_cube.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-a47b1107b5937e1285f5d75d0a7ea2f7' (time: 73,
                                                                band: 3,
                                                                y: 227, x: 163)>
dask.array<getitem, shape=(73, 3, 227, 163), dtype=float64, chunksize=(1, 1, 227, 163), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 2022-01-03...
    id                                       (time) <U24 'S2B_32TPR_20220103_...
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 6.874e+05 ... 6.891e+05
  * y                                        (y) float64 5.044e+06 ... 5.041e+06
    mgrs:latitude_band                       <U1 'T'
    ...                                       ...
    title                                    (band) <U20 'Red (band 4) - 10m'...
    proj:shape                               object {10980}
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0

We might be interested in aggregating our data over periods like week, month, year etc., defining what operation to use to combine the data available in the chosen period.

Using `aggregate_temporal_period` we can achieve this easily:

In [3]:
s2_monthly_min = s2_cube.aggregate_temporal_period(period="month",reducer="median")
s2_monthly_min

Check what happens to the datacube inspecting the resulting xArray object. Now the `time` dimension has 12 labels, one for each month.

In [4]:
s2_monthly_min.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-16b267d96f1130da990e8b62c20f4500' (time: 12,
                                                                band: 3,
                                                                y: 227, x: 163)>
dask.array<stack, shape=(12, 3, 227, 163), dtype=float64, chunksize=(1, 3, 227, 163), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 6.874e+05 ... 6.891e+05
  * y                                        (y) float64 5.044e+06 ... 5.041e+06
    mgrs:latitude_band                       <U1 'T'
    s2:product_type                          <U7 'S2MSI2A'
    mgrs:utm_zone                            int64 32
    ...                                       ...
    proj:shape                               object {10980}
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int64 32632
  * time                                     (time) datetime64[ns] 2022-01-31...
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0